In [2]:
!pip3 install nltk


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import re
import nltk
import string

pd.options.mode.chained_assignment = None

full_df = pd.read_csv('tweet_small.csv')

In [4]:
full_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [5]:
df = full_df[['text']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    93 non-null     object
dtypes: object(1)
memory usage: 876.0+ bytes


In [6]:
print(df) # Apple == apple

                                                 text
0   @AppleSupport causing the reply to be disregar...
1   @105835 Your business means a lot to us. Pleas...
2   @76328 I really hope you all change but I'm su...
3   @105836 LiveChat is online at the moment - htt...
4   @VirginTrains see attached error message. I've...
..                                                ...
88  @105860 I wish Amazon had an option of where I...
89  They reschedule my shit for tomorrow https://t...
90  @105861 Hey Sara, sorry to hear of the issues ...
91  @Tesco bit of both - finding the layout cumber...
92  @105861 If that doesn't help please DM your fu...

[93 rows x 1 columns]


### Перевод к нижнему регистру

In [7]:
df['text_lower'] = df['text'].str.lower()
print(df)

                                                 text  \
0   @AppleSupport causing the reply to be disregar...   
1   @105835 Your business means a lot to us. Pleas...   
2   @76328 I really hope you all change but I'm su...   
3   @105836 LiveChat is online at the moment - htt...   
4   @VirginTrains see attached error message. I've...   
..                                                ...   
88  @105860 I wish Amazon had an option of where I...   
89  They reschedule my shit for tomorrow https://t...   
90  @105861 Hey Sara, sorry to hear of the issues ...   
91  @Tesco bit of both - finding the layout cumber...   
92  @105861 If that doesn't help please DM your fu...   

                                           text_lower  
0   @applesupport causing the reply to be disregar...  
1   @105835 your business means a lot to us. pleas...  
2   @76328 i really hope you all change but i'm su...  
3   @105836 livechat is online at the moment - htt...  
4   @virgintrains see attached erro

### Удаление пунктуации

In [8]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punct(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df['text_wo_punct'] = df['text_lower'].apply(remove_punct)
df.head(10)

,text,text_lower,text_wo_punct
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...
5,"@105836 Have you tried from another device, Mi...","@105836 have you tried from another device, mi...",105836 have you tried from another device miri...
6,"@VirginTrains yep, I've tried laptop too sever...","@virgintrains yep, i've tried laptop too sever...",virgintrains yep ive tried laptop too several ...
7,"@105836 It's working OK from here, Miriam. Doe...","@105836 it's working ok from here, miriam. doe...",105836 its working ok from here miriam does th...
8,@VirginTrains I still haven't heard &amp; the ...,@virgintrains i still haven't heard &amp; the ...,virgintrains i still havent heard amp the numb...
9,@105836 That's what we're here for Miriam 😊 T...,@105836 that's what we're here for miriam 😊 t...,105836 thats what were here for miriam 😊 the ...


### Удаление стоп-слов

In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')
print(', '.join(stopwords.words('english')))

a, about, above, after, again, against, ain, all, am, an, and, any, are, aren, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can, couldn, couldn't, d, did, didn, didn't, do, does, doesn, doesn't, doing, don, don't, down, during, each, few, for, from, further, had, hadn, hadn't, has, hasn, hasn't, have, haven, haven't, having, he, he'd, he'll, her, here, hers, herself, he's, him, himself, his, how, i, i'd, if, i'll, i'm, in, into, is, isn, isn't, it, it'd, it'll, it's, its, itself, i've, just, ll, m, ma, me, mightn, mightn't, more, most, mustn, mustn't, my, myself, needn, needn't, no, nor, not, now, o, of, off, on, once, only, or, other, our, ours, ourselves, out, over, own, re, s, same, shan, shan't, she, she'd, she'll, she's, should, shouldn, shouldn't, should've, so, some, such, t, than, that, that'll, the, their, theirs, them, themselves, then, there, these, they, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, v

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/artemgolubnichiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return ' '.join([word for word in str(text).split(' ') if word not in STOPWORDS])


df['text_wo_stop'] = df['text_wo_punct'].apply(remove_stopwords)
df.head()

,text,text_lower,text_wo_punct,text_wo_stop
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...,applesupport causing reply disregarded tapped ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...,105835 business means lot us please dm name zi...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...,105836 livechat online moment httpstcosy94vtu...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...


### Удаление частых слов

In [11]:
from collections import Counter
cnt = Counter()
for text in df['text_wo_stop'].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('us', 25),
 ('dm', 19),
 ('help', 18),
 ('thanks', 13),
 ('httpstcogdrqu22ypt', 12),
 ('applesupport', 11),
 ('please', 11),
 ('phone', 9),
 ('hi', 9),
 ('ive', 8)]

In [12]:
FREQWORDS = set([w for (w,s) in cnt.most_common(10)])

def remove_freqwords(text):
    return ' '.join([word for word in str(text).split(' ') if word not in FREQWORDS])

df['text_wo_stop_freq'] = df['text_wo_stop'].apply(remove_freqwords)
df.head()

,text,text_lower,text_wo_punct,text_wo_stop,text_wo_stop_freq
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...,applesupport causing reply disregarded tapped ...,causing reply disregarded tapped notification ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...,105835 business means lot us please dm name zi...,105835 business means lot name zip code additi...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...,76328 really hope change im sure wont dont,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...,105836 livechat online moment httpstcosy94vtu...,105836 livechat online moment httpstcosy94vtu...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...,virgintrains see attached error message tried ...


### Удаление редких слов

In [13]:
n_rare_words = 10
RAREWORDS = set([w for (w,s) in cnt.most_common()[:-n_rare_words-1:-1]])
RAREWORDS

{'browser',
 'green',
 'httpstco9281okeebk',
 'including',
 'keen',
 'lee',
 'line',
 'log',
 'slowdown',
 'thin'}

In [14]:
def remove_rarewords(text):
    return ' '.join([word for word in str(text).split(' ') if word not in RAREWORDS])

df['text_wo_stop_freq_rare'] = df['text_wo_stop_freq'].apply(remove_rarewords)
df.head()

,text,text_lower,text_wo_punct,text_wo_stop,text_wo_stop_freq,text_wo_stop_freq_rare
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...,applesupport causing reply disregarded tapped ...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...,105835 business means lot us please dm name zi...,105835 business means lot name zip code additi...,105835 business means lot name zip code additi...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...,76328 really hope change im sure wont dont,76328 really hope change im sure wont dont,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...,105836 livechat online moment httpstcosy94vtu...,105836 livechat online moment httpstcosy94vtu...,105836 livechat online moment httpstcosy94vtu...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...,virgintrains see attached error message ive tr...,virgintrains see attached error message tried ...,virgintrains see attached error message tried ...


### Стемминг

In [15]:
df.drop(['text_lower', 'text_wo_punct', 'text_wo_stop', 'text_wo_stop_freq'], axis=1, inplace=True)

In [16]:
df.head()

,text,text_wo_stop_freq_rare
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...
1,@105835 Your business means a lot to us. Pleas...,105835 business means lot name zip code additi...
2,@76328 I really hope you all change but I'm su...,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu...
4,@VirginTrains see attached error message. I've...,virgintrains see attached error message tried ...


In [17]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

df['text_stemmed'] = df['text_wo_stop_freq_rare'].apply(stem_words)
df.head()

,text,text_wo_stop_freq_rare,text_stemmed
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,caus repli disregard tap notif keyboard opened😡😡😡
1,@105835 Your business means a lot to us. Pleas...,105835 business means lot name zip code additi...,105835 busi mean lot name zip code addit detai...
2,@76328 I really hope you all change but I'm su...,76328 really hope change im sure wont dont,76328 realli hope chang im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu...,105836 livechat onlin moment httpstcosy94vtu8k...
4,@VirginTrains see attached error message. I've...,virgintrains see attached error message tried ...,virgintrain see attach error messag tri leav v...


### Обработка русского языка

In [18]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')
wrds = ['рентгеноэлектрокардиографический', 'трансцендентный', 'автостеклоподъемники',  'электростеклоподъемники', 'длинношеее', 'урегулирование',
'воспользовавшийся', 'сиреневенький', 'переосвидетельствовать', 'человеконенавистничество', 'высокопревосходительство',
'защищающихся']
for w in wrds:
    print(stemmer.stem(w))

рентгеноэлектрокардиографическ
трансцендентн
автостеклоподъемник
электростеклоподъемник
длинноше
урегулирован
воспользова
сиреневеньк
переосвидетельствова
человеконенавистничеств
высокопревосходительств
защища


### Лемматизация

In [22]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

df['text_lemmatized'] = df['text'].apply(lemmatize_words)
df.head(10)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/artemgolubnichiy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,text_wo_stop_freq_rare,text_stemmed,text_lemmatized
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,caus repli disregard tap notif keyboard opened😡😡😡,@AppleSupport causing the reply to be disregar...
1,@105835 Your business means a lot to us. Pleas...,105835 business means lot name zip code additi...,105835 busi mean lot name zip code addit detai...,@105835 Your business mean a lot to us. Please...
2,@76328 I really hope you all change but I'm su...,76328 really hope change im sure wont dont,76328 realli hope chang im sure wont dont,@76328 I really hope you all change but I'm su...
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu...,105836 livechat onlin moment httpstcosy94vtu8k...,@105836 LiveChat is online at the moment - htt...
4,@VirginTrains see attached error message. I've...,virgintrains see attached error message tried ...,virgintrain see attach error messag tri leav v...,@VirginTrains see attached error message. I've...
5,"@105836 Have you tried from another device, Mi...",105836 tried another device miriam mm,105836 tri anoth devic miriam mm,"@105836 Have you tried from another device, Mi..."
6,"@VirginTrains yep, I've tried laptop too sever...",virgintrains yep tried laptop several times pa...,virgintrain yep tri laptop sever time past wee...,"@VirginTrains yep, I've tried laptop too sever..."
7,"@105836 It's working OK from here, Miriam. Doe...",105836 working ok miriam link httpstco0m2mph15...,105836 work ok miriam link httpstco0m2mph15eh mm,"@105836 It's working OK from here, Miriam. Doe..."
8,@VirginTrains I still haven't heard &amp; the ...,virgintrains still havent heard amp number im ...,virgintrain still havent heard amp number im d...,@VirginTrains I still haven't heard &amp; the ...
9,@105836 That's what we're here for Miriam 😊 T...,105836 thats miriam 😊 team send email shortly hp,105836 that miriam 😊 team send email shortli hp,@105836 That's what we're here for Miriam 😊 Th...


In [25]:
df['eq'] = df['text'] == df['text_lemmatized']
df.head(10)

,text,text_wo_stop_freq_rare,text_stemmed,text_lemmatized,eq
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,caus repli disregard tap notif keyboard opened😡😡😡,@AppleSupport causing the reply to be disregar...,True
1,@105835 Your business means a lot to us. Pleas...,105835 business means lot name zip code additi...,105835 busi mean lot name zip code addit detai...,@105835 Your business mean a lot to us. Please...,False
2,@76328 I really hope you all change but I'm su...,76328 really hope change im sure wont dont,76328 realli hope chang im sure wont dont,@76328 I really hope you all change but I'm su...,True
3,@105836 LiveChat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu...,105836 livechat onlin moment httpstcosy94vtu8k...,@105836 LiveChat is online at the moment - htt...,True
4,@VirginTrains see attached error message. I've...,virgintrains see attached error message tried ...,virgintrain see attach error messag tri leav v...,@VirginTrains see attached error message. I've...,False
5,"@105836 Have you tried from another device, Mi...",105836 tried another device miriam mm,105836 tri anoth devic miriam mm,"@105836 Have you tried from another device, Mi...",True
6,"@VirginTrains yep, I've tried laptop too sever...",virgintrains yep tried laptop several times pa...,virgintrain yep tri laptop sever time past wee...,"@VirginTrains yep, I've tried laptop too sever...",False
7,"@105836 It's working OK from here, Miriam. Doe...",105836 working ok miriam link httpstco0m2mph15...,105836 work ok miriam link httpstco0m2mph15eh mm,"@105836 It's working OK from here, Miriam. Doe...",True
8,@VirginTrains I still haven't heard &amp; the ...,virgintrains still havent heard amp number im ...,virgintrain still havent heard amp number im d...,@VirginTrains I still haven't heard &amp; the ...,True
9,@105836 That's what we're here for Miriam 😊 T...,105836 thats miriam 😊 team send email shortly hp,105836 that miriam 😊 team send email shortli hp,@105836 That's what we're here for Miriam 😊 Th...,False


In [24]:
lemmatizer.lemmatize('running', pos='v')

'run'

### Регулярные выражения

#### Поиск даты

In [31]:
import re
text = 'сегодня 31.04.2025'
pattern = r'(0[1-9]|[12][0-9]|3[01])\.(0[1-9]|1[0-2])\.(19|20)\d{2}'
match_date = re.search(pattern, text)
print(match_date)


<re.Match object; span=(8, 18), match='31.04.2025'>


In [39]:
text = 'проверка ip адресса 217.2.2.199'
pattern = r'(?:(?:25[0-5]|2[0-4]\d|1\d{2}|[1-9]?\d)\.){3}(?:25[0-5]|2[0-4]\d|1\d{2}|[1-9]?\d)'
match_date = re.search(pattern, text)
print(match_date)

<re.Match object; span=(20, 31), match='217.2.2.199'>


In [42]:
import re
text = "foo123"
match = re.search(r"(foo)\d+", text)
print(match.group(1))  # → 'foo'


match = re.search(r"(?:foo)\d+", text)
print(match.groups())  # → ()

foo
()


1. Найти все слова, начинающиеся с большой буквы
2. Извлечь все email-адреса
3. Найти все слова из 4 букв
4. Извлечь все телефонные номера
5. Найти все слова, содержащие только буквы
6. Найти все числа с плавающей точкой
7. Извлечь все доменные имена
8. Найти все ссылки
9. Найти все слова, где есть две одинаковые буквы подряд
10. Найти повторяющиеся слова подряд

In [46]:
pat = r'\b[A-ZА-Я][a-zа-яё]*\b'

text = 'Проверка IP Адресса 217.2.2.199'

match_date = re.findall(pat, text)
print(match_date)

['Проверка', 'Адресса']


In [47]:
text = "Артем Голубничий отличный преподаватель!"
pattern = r'\b[А-ЯЁ][а-яё]*\b'
matches = re.findall(pattern, text)
print(matches) 

['Артем', 'Голубничий']


In [48]:
pattern = r'[a-zA-Z0-9._-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
text = "sdsdvs@mail.ru sdsdvsmail.ru sdsdvs@mailru sdsdvs@mail.u ывсым@mail.ru"
matches = re.findall(pattern, text)
print(matches) 

['sdsdvs@mail.ru']


In [50]:
pattern = r'\b\w{4}\b'
text = "Набор слов каждое из 4 букв"
matches = re.findall(pattern, text)
print(matches) 

['слов', 'букв']


In [ ]:
pattern = r'\+?[1-9][0-9]{6,10}'
text = "Текст в виде номера 9202189001 678234 89"
matches = re.findall(pattern, text)
print(matches) 

[]


In [54]:
pattern = r'\b[а-яА-ЯёЁa-zA-Z]+\b'
text = "Текст в виде номера3 9202189001 678234 89"
matches = re.findall(pattern, text)
print(matches) 

['Текст', 'в', 'виде']


In [58]:
pattern = r'-?\d+\.\d*|\.\d+'
text = "9202.189001 678234.0 -89.09 .35 .8"
matches = re.findall(pattern, text)
print(matches)

['9202.189001', '678234.0', '-89.09', '.35', '.8']


In [62]:
pattern = r'[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
text = "edu.mail.ru site.com name.info"
matches = re.findall(pattern, text)
print(matches)

['edu.mail.ru', 'site.com', 'name.info']


In [ ]:
pattern = r'(http|https|ftp){1}:{1}/{2}(\w+\.)+\w{2,3}'
text = "http://site.com text ftp://edu.sitw.com"
matches = re.findall(pattern, text)
print(matches)

[]


In [64]:
pattern = r'\b\w*(\w)\1\w*\b'
text = "Текст в виде аан роон"
matches = re.findall(pattern, text)
print(matches) 

['а', 'о']


In [ ]:
text = "Молл, класс, стул, окно, лама, кроссворд, мама, пресс"
matches = re.findall(r"\b\w*(\w)\1\w*\b", text)
print("Слова с двойными буквами:")

words = re.findall(r"\b\w*(\w)\1\w*\b", text)
all_words = re.findall(r"\b\w+\b", text)
double_letter_words = [word for word in all_words if re.search(r"(.)\1", word)]
print(double_letter_words)

Слова с двойными буквами:
['Молл', 'класс', 'кроссворд', 'пресс']
